In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()

df = pd.concat((
    pd.read_csv('./data/October_missions_full.csv'),
    pd.read_csv('./data/November_1stW_missions_full.csv'),
), ignore_index=True)

df['mission'] = df['type'] + '_' + df['target'].astype(str)

df = df[['user', 'mission', 'createdAtT', 'type', 'target', 'completed', 'performance']]
df['createdAtT'] = pd.to_datetime(df['createdAtT'], unit='ms').dt.date
df['user'] = df['user'].astype('category').cat.codes
df['mission'] = df['mission'].astype('category')
df['type'] = df['type'].astype('category')

df.sort_values(by=['createdAtT', 'user'], inplace=True, ignore_index=True)
df

In [ ]:
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
df['completed'].value_counts(normalize=True).plot(kind='bar')

plt.subplot(1, 2, 2)
df['performance'].plot.hist(bins=20)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

df.groupby('mission', observed=True).size().plot(kind='bar')

In [ ]:
plt.figure(figsize=(10, 6))

df.groupby('type', observed=True).size().plot(kind='bar')

In [ ]:
def sequences(x: pd.Series, df: pd.DataFrame, day_window=3):
    ts = x['createdAtT']
    ts_start = ts - pd.Timedelta(days=day_window)

    user_df: pd.DataFrame = df[(df['user'] == x['user']) & (df['createdAtT'] >= ts_start) & (df['createdAtT'] < ts)]
    user_df = user_df.sort_values(by='createdAtT', ascending=True)
    return pd.Series({
        'id': str(x['user']) + '_' + str(ts),
        'ts': ts,
        'mission': x['mission'],
        'ID': x['mission_id'],
        'history': user_df[['mission_id', 'ok']].astype(int).values
    })

def get_negatives(x: pd.Series):
    neg = missions - set(x)
    return [list(neg)] * len(x)

df['ok'] = df['performance'].apply(lambda x: 0.7 <= x <= 1.05)
df['mission_id'] = df['mission'].cat.codes
missions = set(df['mission_id'])

sequences_df: pd.DataFrame = df[df['ok']].progress_apply(sequences, axis=1, args=(df,))
sequences_df['negatives'] = sequences_df.groupby('id')['ID'].transform(get_negatives)

sequences_df

In [ ]:
sequences_df['ts'] = pd.to_datetime(sequences_df['ts'])
sequences_df = sequences_df[sequences_df['history'].apply(len) > 0]
train_df = sequences_df[sequences_df['ts'] < pd.Timestamp('2024-11-01')]
test_df = sequences_df[sequences_df['ts'] >= pd.Timestamp('2024-11-01')]

display(train_df)
display(test_df)

In [ ]:
train_df.to_pickle('./out/train.pkl')
test_df.to_pickle('./out/test.pkl')